In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [2]:
# hyperparameters
batch_size = 32 # how many independent sequences will we process in parallel?
block_size = 128 # what is the maximum context length for predictions?
max_iters = 2000
eval_interval = 200
learning_rate = 6e-3
eval_iters = 20
n_embed = 256
n_layer = 2

device = 'cuda' if torch.cuda.is_available() else 'cpu'
#torch.manual_seed(42)

In [3]:
data_dir = 'data/tinyshakespeare/'
with open(data_dir + 'input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [4]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [5]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("i like ice cream"))
print(decode(encode("i like ice cream")))

[47, 1, 50, 47, 49, 43, 1, 47, 41, 43, 1, 41, 56, 43, 39, 51]
i like ice cream


In [6]:
# train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [7]:
# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [8]:
@torch.no_grad()
def estimate_loss():
    model.eval()
    out = {}
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, _, loss = model(X, model.init_hidden(batch_size), Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [9]:
class simpleRNN(nn.Module):
    def __init__(self, n_embed, n_hidden, n_layer, tie_weights=True):
        super().__init__()
        self.n_embed = n_embed
        self.n_hidden = n_hidden
        self.n_layer = n_layer

        self.encoder = nn.Embedding(vocab_size, n_embed)
        self.rnn = nn.RNN(n_embed, n_hidden, n_layer, batch_first=True)
        self.decoder = nn.Linear(n_hidden, vocab_size)
        
        if tie_weights:
            if n_embed != n_hidden:
                raise ValueError('When using the tied flag, embed size must be equal to hidden size')
            self.decoder.weight = self.encoder.weight
    
    def forward(self, idx, hx, targets=None):
        B, T = idx.shape
        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.encoder(idx) # (B, T, C)
        y, h = self.rnn(tok_emb, hx) # (B, T, C)
        logits = self.decoder(y) # (B, T, vocab_size)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, h, loss
    
    def init_hidden(self, batch_size):
        # first hidden state is all zeros
        return torch.zeros(self.n_layer, batch_size, self.n_hidden).to(device)
    
    @torch.no_grad()
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, hidden, loss = self(idx_cond, self.init_hidden(1))
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [10]:
model = simpleRNN(n_embed, n_embed, n_layer).to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

In [11]:
for iteration in range(max_iters):
    
    # every once in a while evaluate the loss on train and val sets
    if iteration % eval_interval == 0 or iteration == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iteration}/{max_iters}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
    
    # sample a batch of data
    xb, yb = get_batch('train')
    
    # init hidden state for first input
    hb = model.init_hidden(batch_size)
    
    # evaluate the loss
    optimizer.zero_grad()
    logits, hb, loss = model(xb, hb, targets=yb)
    loss.backward()
    optimizer.step()

step 0/2000: train loss 12.9704, val loss 12.9226
step 200/2000: train loss 2.0886, val loss 2.1652
step 400/2000: train loss 1.8374, val loss 1.9788
step 600/2000: train loss 1.7604, val loss 1.9314
step 800/2000: train loss 1.6878, val loss 1.8891
step 1000/2000: train loss 1.6465, val loss 1.8376
step 1200/2000: train loss 1.6155, val loss 1.8434
step 1400/2000: train loss 1.6047, val loss 1.8082
step 1600/2000: train loss 1.5871, val loss 1.7939
step 1800/2000: train loss 1.5461, val loss 1.7601
step 1999/2000: train loss 1.5323, val loss 1.7730


In [12]:
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(model.generate(context, max_new_tokens=500)[0].tolist()))


It make.
Ay, I do pleasure.
And down sworn how!

DUKE VINCENTIO:
Hath the lightions.
Good doge, were Edward to me.

AUFIDIUS:
Fitted Tow, fair youngle liefelless your lord.

HARh:
How long and you have a pass-plight by long gentleman many kissage, which 'lave a difment. Too waitor, with heasts
You worn'd our dearous mind
In aire leave leave,
Some the mine inselso'd seed man kiding dream would awhate, he, to in night of it off sose with foroutair of us arm not with my Beous tender appery
joiful t
